## Group Project Prososal (group 11) 

In [15]:
library(tidyverse)
library(testthat)
library(digest)
library(repr)
library(tidymodels)

## 1. Title : Credit Card Fraud Data Analysis Report



## 2. Introduction:

**1) Background Information** 

As digital payments boomed in recent years with the rapid development of the Internet, people enjoy the fast and convenient transaction service but also face an increasing number of cyber criminals alongside. The Data Breach Index announced that over five million records are being stolen on a daily basis, which indicates that fraud is very common in our daily life. However, detection of fraud is challenging since trillions of card transactions happen per day. 


**2) Research Question**

Based on the analysis, can we predict fraudulent transaction using **<font color=red> distance from home and ratio to median purchase price variables to prevent future crime?

**3) Dataset description**

We will be using a dataset called Credit Card Fraud made by Dhanush Narayanan from https://www.kaggle.com/datasets/dhanushnarayananr/credit-card-fraud. The dataset is in a “Comma Seperated Value” (csv) file format and contains eight columns, which are: 
distance_from_home : the distance from home where the transaction happened
distance_from_last_transaction : the distance from last transaction happened.
ratio_to_median_purchase_price  :  Ratio of purchased price transaction to median purchase price.
repeat_retailer : whether the transaction happened from same retailer or not
used_chip : whether the transaction is through chip (credit card) or not
used_pin_number : whether the transaction happened by using PIN number or not
online_order : whether the transaction is an online order or not
fraud : whether the transaction is fraudulent or not


## 3. Preliminary exploratory data analysis:

Demonstrate that the dataset can be read from the web into R 
Clean and wrangle your data into a tidy format
Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

In [ ]:
#read the file
data <- read.csv("data/card_transdata.csv") %>%
                mutate(repeat_retailer = as_factor(repeat_retailer)) %>%
                mutate(used_chip = as_factor(used_chip)) %>%
                mutate(used_pin_number = as_factor(used_pin_number)) %>%
                mutate(fraud = as_factor(fraud)) 
head(data, 10)

In [ ]:
card_data <- select(data, distance_from_home, ratio_to_median_purchase_price, fraud )
head(card_data, 10)

In [ ]:
set.seed(1)
## creat training data for card_data

card_split <- initial_split(card_data, prop = 0.75, strata = fraud)  
card_train <- training(card_split) 
card_test <- testing(card_split)

head(card_train, 10)

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10) 

card_plot <- ggplot(card_train, aes(x = distance_from_home , y =  ratio_to_median_purchase_price, color = fraud)) +
            geom_point() +
            labs( x = "Distance from home (km)" , y = "Purchase price/ median purchase price" , color = "is the transaction fraudulent?") +
            theme(text = element_text(size = 20))

card_plot

## 4. Methods:
Explain how you will conduct either your data analysis and which variables/columns you will use. 

Note - you do not need to use all variables/columns that exist in the raw data set.

In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?

Describe at least one way that you will visualize the results

In [ ]:
#Find the correlation between fraud and other variables
cor(card_data)

## 5. Expected outcomes and significance:
What do you expect to find?

What impact could such findings have?

What future questions could this lead to?